In [218]:
import pandas as pd
import os
cwd = os.getcwd()

ROOT = os.path.join(cwd, '../')
DATA_FOLDER = os.path.join(ROOT, 'data/')

df = pd.read_pickle(os.path.join(DATA_FOLDER, 'saeb_pb_2017-2007_fe.pkl'))

df = df.dropna()
print(len(df))
df.head()

232882


,id_aluno,disciplina,proficiencia,erro_padrao,desempenho_aluno,desempenho_aluno_cat,faz_licao_casa_2.0,reprovacao_0.0,reprovacao_1.0,pai_sabe_ler_escrever_1.0,leitura_revista_comportamento_0.0,leitura_jornais_0.0,escolaridade_pai_4.0,mae_sabe_ler_escrever_1.0,leitura_internet_2.0,tempo_lazer_1.0,escolaridade_mae_4.0,gosta_estudar_disciplina_0.0,gosta_estudar_disciplina_1.0
8086,23061434,LP,0.665701,0.312872,Proficiente,2,True,True,False,False,True,True,False,True,True,True,False,False,True
10602,23065563,LP,-1.353424,0.484866,Insuficiente,0,True,False,True,True,False,False,False,False,True,True,False,False,True
13975,23065563,MT,-1.652930,0.564617,Insuficiente,0,True,False,True,True,False,False,False,False,True,True,False,False,True
17847,23065552,MT,-1.099605,0.467812,Insuficiente,0,True,True,False,False,False,False,False,True,False,False,False,False,True
20664,23095249,MT,-1.262341,0.512487,Insuficiente,0,False,False,True,True,False,False,False,True,True,False,False,True,False


In [219]:
print(df.columns)

Index(['id_aluno', 'disciplina', 'proficiencia', 'erro_padrao',
       'desempenho_aluno', 'desempenho_aluno_cat', 'faz_licao_casa_2.0',
       'reprovacao_0.0', 'reprovacao_1.0', 'pai_sabe_ler_escrever_1.0',
       'leitura_revista_comportamento_0.0', 'leitura_jornais_0.0',
       'escolaridade_pai_4.0', 'mae_sabe_ler_escrever_1.0',
       'leitura_internet_2.0', 'tempo_lazer_1.0', 'escolaridade_mae_4.0',
       'gosta_estudar_disciplina_0.0', 'gosta_estudar_disciplina_1.0'],
      dtype='object')


In [220]:
# distribuição das classes
df['desempenho_aluno'].value_counts(dropna=False)

desempenho_aluno
Básico          124830
Insuficiente     78056
Proficiente      26662
Avançado          3334
Name: count, dtype: int64

In [221]:
exclude = ['ano', 'sigla_uf', 'situacao_censo', 'disciplina', 'proficiencia', 'erro_padrao', 'proficiencia_saeb',
       'erro_padrao_saeb', 'sexo', 'raca_cor','ano_nascimento', 'rede', 'id_escola', 'serie', 'turno', 'escola_publica', 'desempenho_aluno', 'proficiencia']

exclude = [col for col in df.columns if col in exclude]

In [222]:
df = df[df['desempenho_aluno_cat'] != 2]

In [223]:
df.loc[:, 'desempenho_aluno_cat'] = df['desempenho_aluno_cat'].apply(lambda x: 1 if x == 3 else 0)

In [224]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from imblearn.under_sampling import RandomUnderSampler

from IPython.display import clear_output

clear_output()

In [237]:
# a lógica é a seguinte:
# analisaremos linhas no dataset com os mesmos valores de features, mas com diferentes valores de target
# analisando os tamanhos de cada grupo, consideraremos o erro aquele grupo que tiver menos instâncias
# e removeremos todas as instâncias desse grupo

In [225]:
X = df.drop(columns=exclude + ['desempenho_aluno_cat'])
y = df['desempenho_aluno_cat']

In [226]:
res = RandomUnderSampler(sampling_strategy='majority')
X_res, y_res = res.fit_resample(X, y)

In [227]:
y_res.value_counts()

desempenho_aluno_cat
0    3334
1    3334
Name: count, dtype: int64

In [228]:
df = pd.concat([X_res, y_res], axis=1)
df.head()

,id_aluno,faz_licao_casa_2.0,reprovacao_0.0,reprovacao_1.0,pai_sabe_ler_escrever_1.0,leitura_revista_comportamento_0.0,leitura_jornais_0.0,escolaridade_pai_4.0,mae_sabe_ler_escrever_1.0,leitura_internet_2.0,tempo_lazer_1.0,escolaridade_mae_4.0,gosta_estudar_disciplina_0.0,gosta_estudar_disciplina_1.0,desempenho_aluno_cat
38709,17486608,True,False,True,False,False,False,False,False,False,False,False,True,False,0
75835,17542236,False,True,False,True,False,True,False,True,True,False,False,False,True,0
31759,12145735,True,False,True,True,False,False,False,False,False,False,False,False,True,0
13020,4986194,True,True,False,False,False,False,False,False,False,False,False,False,True,0
28547,23045906,False,True,False,True,False,True,False,True,False,False,False,True,False,0


In [229]:
feature_columns = df.columns[1:-1]
print(feature_columns)

Index(['faz_licao_casa_2.0', 'reprovacao_0.0', 'reprovacao_1.0',
       'pai_sabe_ler_escrever_1.0', 'leitura_revista_comportamento_0.0',
       'leitura_jornais_0.0', 'escolaridade_pai_4.0',
       'mae_sabe_ler_escrever_1.0', 'leitura_internet_2.0', 'tempo_lazer_1.0',
       'escolaridade_mae_4.0', 'gosta_estudar_disciplina_0.0',
       'gosta_estudar_disciplina_1.0'],
      dtype='object')


In [230]:
# show dataframe with all X.columns value equal to False
df[(df[feature_columns] == 0).all(axis=1)]['desempenho_aluno_cat'].value_counts()

desempenho_aluno_cat
0    29
1    12
Name: count, dtype: int64

In [231]:
# show dataframe with all X.columns value equal to False
df[(df[feature_columns] == 1).all(axis=1)]['desempenho_aluno_cat'].value_counts()

Series([], Name: count, dtype: int64)

In [232]:
values_oi = [1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1]

df[(df[feature_columns] == values_oi).all(axis=1)]['desempenho_aluno_cat'].value_counts()

desempenho_aluno_cat
1    36
0     6
Name: count, dtype: int64

In [233]:
# all combinations of 0 and 1 for 13 features
from itertools import product
import numpy as np
combinations = list(product([0, 1], repeat=13))

In [235]:
for comb in combinations:
    #print(df[(df[feature_columns] == comb).all(axis=1)]['desempenho_aluno_cat'].value_counts())
    group_counts = df[(df[feature_columns] == comb).all(axis=1)]['desempenho_aluno_cat'].value_counts()
    # drop the least frequent class
    if len(group_counts) > 1:
        print('dropping')
        print(comb)
        df = df.drop(df[(df[feature_columns] == comb).all(axis=1)].index)
        print(len(df))
    print()

dropping
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
6626

dropping
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1)
6619







dropping
(0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0)
6613
























dropping
(0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0)
6580




dropping
(0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0)
6566





























































































































































































































































































































































































dropping
(0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1)
6563

















dropping
(0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0)
6559























































































dropping
(0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1)
6554























dropping
(0, 0, 0, 1, 0,

In [236]:
df['desempenho_aluno_cat'].value_counts()

desempenho_aluno_cat
0    929
1    244
Name: count, dtype: int64

In [238]:
df.to_pickle(os.path.join(DATA_FOLDER, 'saeb_pb_2017-2007_fe_balanced.pkl'))